<a href="https://colab.research.google.com/github/Harisudhan23/NLP/blob/NLP/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install transformers datasets scikit-learn nltk rouge-score torch
!pip install datasets
!pip install load_metric
!pip install evaluate

ERROR: Could not find a version that satisfies the requirement load_metric (from versions: none)
ERROR: No matching distribution found for load_metric
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'cleaned_reviews.csv'

try:
    data = pd.read_csv(file_path)
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the file path.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty. Please provide a valid CSV file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Check if the dataset contains data
if 'data' in locals() and not data.empty:
    # Display the first few rows to understand the dataset structure
    print("First few rows of the dataset:")
    print(data.head())

    # Display dataset information
    print("\nDataset information:")
    print(data.info())

    # Check for missing values
    print("\nChecking for missing values:")
    print(data.isnull().sum())
else:
    print("The dataset is empty or could not be loaded.")


Dataset loaded successfully!
First few rows of the dataset:
  sentiments                                     cleaned_review  \
0   positive  i wish would have gotten one earlier love it a...   
1    neutral  i ve learned this lesson again open the packag...   
2    neutral          it is so slow and lags find better option   
3    neutral  roller ball stopped working within months of m...   
4    neutral  i like the color and size but it few days out ...   

   cleaned_review_length  review_score  
0                     19           5.0  
1                     88           1.0  
2                      9           2.0  
3                     12           1.0  
4                     21           1.0  

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17340 entries, 0 to 17339
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sentiments             17340 non-null  object 
 1   

<ipython-input-16-ed98221759e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_review'] = data['cleaned_review'].fillna('')


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode target labels (sentiments) into numerical values
label_encoder = LabelEncoder()
data['sentiments_encoded'] = label_encoder.fit_transform(data['sentiments'])

# Split the dataset into training and test sets
X = data['cleaned_review']
y = data['sentiments_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display the split sizes and a sample
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((13872,), (3468,), (13872,), (3468,))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Replace NaN values in training and test sets
X_train = X_train.fillna('')
X_test = X_test.fillna('')

# Transform text data using Bag-of-Words
vectorizer_bow = CountVectorizer(max_features=5000)  # Limit to top 5000 words
X_train_bow = vectorizer_bow.fit_transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)

# Train a Logistic Regression classifier
clf_bow = LogisticRegression(max_iter=1000, random_state=42)
clf_bow.fit(X_train_bow, y_train)

# Predict and evaluate the model
y_pred_bow = clf_bow.predict(X_test_bow)
report_bow = classification_report(y_test, y_pred_bow, target_names=label_encoder.classes_)

print("Bag-of-Words Classification Report:\n")
print(report_bow)


Bag-of-Words Classification Report:

              precision    recall  f1-score   support

    negative       0.75      0.57      0.65       307
     neutral       0.83      0.87      0.85      1261
    positive       0.93      0.93      0.93      1900

    accuracy                           0.88      3468
   macro avg       0.84      0.79      0.81      3468
weighted avg       0.87      0.88      0.87      3468



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load and preprocess the dataset
file_path = 'cleaned_reviews.csv'
data = pd.read_csv(file_path)

# Drop rows with missing or invalid data and ensure text is string
data = data.dropna(subset=['cleaned_review', 'sentiments'])
data['cleaned_review'] = data['cleaned_review'].astype(str)

# Encode target labels
label_encoder = LabelEncoder()
data['sentiments_encoded'] = label_encoder.fit_transform(data['sentiments'])

# Split the dataset
X = data['cleaned_review']
y = data['sentiments_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Path to GloVe file
glove_file = 'glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_file)

# Convert text to GloVe embeddings
def text_to_embedding(text, embeddings_index, embed_dim=100):
    words = text.split()
    embeddings = []
    for word in words:
        embedding = embeddings_index.get(word)
        if embedding is not None and embedding.shape == (embed_dim,):  # Validate shape
            embeddings.append(embedding)

    if embeddings:  # Ensure embeddings list is not empty
        embeddings = np.array(embeddings)  # Convert to consistent NumPy array
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(embed_dim)  # Return zero vector if no valid embeddings found

# Transform training and testing data
X_train_glove = np.vstack([text_to_embedding(text, embeddings_index) for text in X_train])
X_test_glove = np.vstack([text_to_embedding(text, embeddings_index) for text in X_test])

# Train Logistic Regression classifier
clf_glove = LogisticRegression(max_iter=1000, random_state=42)
clf_glove.fit(X_train_glove, y_train)

# Evaluate the model
y_pred_glove = clf_glove.predict(X_test_glove)
report_glove = classification_report(y_test, y_pred_glove, target_names=label_encoder.classes_)

print("GloVe Embeddings Classification Report:\n")
print(report_glove)


GloVe Embeddings Classification Report:

              precision    recall  f1-score   support

    negative       0.51      0.06      0.11       307
     neutral       0.61      0.63      0.62      1260
    positive       0.76      0.85      0.80      1901

    accuracy                           0.70      3468
   macro avg       0.63      0.51      0.51      3468
weighted avg       0.68      0.70      0.67      3468



In [ ]:
pip install transformers


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['cleaned_review'], padding="max_length", truncation=True)

train_encodings = tokenizer(list(X_train), padding=True, truncation=True, max_length=128, return_tensors="pt")
test_encodings = tokenizer(list(X_test), padding=True, truncation=True, max_length=128, return_tensors="pt")

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings,
    eval_dataset=test_encodings
)

trainer.train()

from sklearn.metrics import classification_report

# Example for BoW:
print(classification_report(y_test, y_pred_bow, target_names=label_encoder.classes_))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [ ]:
!pip install rouge_score

ERROR: Could not find a version that satisfies the requirement rouge_scorer (from versions: none)
ERROR: No matching distribution found for rouge_scorer


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = [summarizer(text, max_length=50, min_length=25, do_sample=False) for text in X_test[:5]]
print(summaries)

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
for i, text in enumerate(X_test[:5]):
    print(scorer.score(text, summaries[i]))
    print(sentence_bleu([text.split()], summaries[i]['summary_text'].split()))


Device set to use cpu
Your max_length is set to 50, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but your input_length is only 7. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 50, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 50, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max

[[{'summary_text': " broken right after ran out of return window came the narrow keep moving and linking event when is using or not using it don't know what problem with it."}], [{'summary_text': 'CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots.'}], [{'summary_text': ' works sometimes very unreliable waited too long to return due to trying different things so that on me had to purchase different keyboard.'}], [{'summary_text': 'i love my new keyboard like how it turns off when not in use and is wireless and does not need battery.'}], [{'summary_text': '" surround sound in the backyard and around our pool" "My favorite part is we can synch all there and have surround sound" "We have three my husband and each of my teenage boys"'}]]


AttributeError: 'list' object has no attribute 'lower'

# New Section